<a href="https://colab.research.google.com/github/cgsnider/Restaurant-Recs/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The recommender model.
Designed to be run on Colab.

Note: Requires access to your google drive and for the teams folder "ML" to be located in the root directory of your drive (cannot be inside another folder)


In [2]:
!pip install lightfm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from lightfm.data import Dataset

/Users/rajkumarnattamaisubramanian/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [4]:
reviews_df = pd.read_json('/Users/rajkumarnattamaisubramanian/Downloads/data/cleaned/yelp_academic_dataset_review.json')

In [6]:
reviews_df['business_id']

0          XQfwVwDr-v0ZS3_CbbE5Xw
2          YjUWPpI6HXG530lwP-fb2A
3          kxX2SOes4o-D3ZQBkiMRfA
9          gebiRewfieSdtt17PTW6Zg
11         EQ-TZ2eeD_E0BHuvoaeG5Q
                    ...          
6990270    1_BVWDzi5cVqWxNe9bOMMQ
6990272    e_E-jq9mwm7wk75k7Yi-Xw
6990274    X4mouE_cMiwbfyCPZ_K-FA
6990278    Rr9kKArrMhSLVE9a53q-aA
6990279    VAeEXLbEcI9Emt9KGYq9aA
Name: business_id, Length: 4104995, dtype: object

In [7]:
#Builds the mapping for restaurant's id (business_id) and customer's id (user_id)
dataset = Dataset()
dataset.fit(reviews_df.user_id.unique(), reviews_df.business_id.unique())

In [8]:
num_users, num_items = dataset.interactions_shape()
print(f'num_users: {num_users}, num_items: {num_items}')

num_users: 1333169, num_items: 44582


In [9]:
#Runtime ~4min
interactions, weights = dataset.build_interactions(((review.user_id, review.business_id) for i, review in reviews_df.iterrows()))

In [10]:
reducedDataset = pd.read_json('/Users/rajkumarnattamaisubramanian/Downloads/data/cleaned/yelp_academic_dataset_restaurant_reduced.json')

In [11]:
reducedDataset

,buisness_id,stars,review_count,postal_code,street,businessacceptscreditcards,restaurantsdelivery,restaurantspricerange2,bikeparking,outdoorseating,...,intimate,hipster,classy,trendy,casual,restaurantsgoodforgroups,alcohol,restaurantsattire,noiselevel,RestaurantsAttire
0,MTSW4McQd7CbVtyjqoe9mw,4.0,80,19107,True,False,False,1,True,False,...,False,False,False,False,True,True,free,none,average,none
1,mWMc6_wTdE0EUBKIGXDVfA,4.5,13,18054,False,True,True,2,True,False,...,False,False,False,False,True,True,no,none,average,none
2,CF33F8-E6oudUQ46HnavjQ,2.0,6,37015,False,True,True,1,False,True,...,False,False,False,False,True,True,no,casual,average,casual
3,bBDDEgkFA1Otx9Lfe7BZUQ,1.5,10,37207,False,True,True,1,False,True,...,False,False,False,False,True,False,no,casual,average,casual
4,eEOYSgkmpB90uNA7lDOMRA,4.0,10,33602,False,True,True,2,False,False,...,False,False,False,False,False,True,no,none,average,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44577,l9eLGG9ZKpLJzboZq-9LRQ,3.0,11,19018,False,True,True,1,True,False,...,False,False,False,False,True,True,no,none,average,none
44578,cM6V90ExQD6KMSU3rRB5ZA,4.0,33,83704,False,True,False,1,True,True,...,False,False,False,False,True,True,free,casual,loud,casual
44579,1jx1sfgjgVg0nM6n3p0xWA,4.5,41,85737,False,True,False,2,True,True,...,False,False,False,False,True,True,free,none,average,none
44580,WnT9NIzQgLlILjPT0kEcsQ,4.5,35,19147,True,True,True,2,True,False,...,False,False,False,False,True,True,free,casual,average,casual


In [37]:
features = list(reducedDataset.columns)
features = features[1::]
features
reducedDataset1 = reducedDataset[features]
reducedDataset1List = [tuple(x) for x in reducedDataset1.values]
reducedTuple = tuple(zip(reducedDataset['buisness_id'],reducedDataset1List))

In [41]:
reducedTuple[0]

('MTSW4McQd7CbVtyjqoe9mw',
 (4.0,
  80,
  '19107',
  True,
  False,
  False,
  '1',
  True,
  False,
  'free',
  True,
  False,
  True,
  False,
  False,
  False,
  False,
  True,
  True,
  'free',
  'none',
  'average',
  'none'))

In [38]:
dataset.fit(reviews_df.user_id.unique(), reducedDataset.buisness_id.unique(),item_features=reducedTuple)

In [39]:
item_features = dataset.build_item_features(reducedTuple)

ValueError: Feature 4.0 not in feature mapping. Call fit first.